In [1]:
import os
import pandas as pd
import numpy as np
import re
from __future__ import division
import math
from scipy.spatial import ConvexHull
import scipy
from configparser import ConfigParser
import glob
# from simba.rw_dfs import *

In [2]:
in_dir = '../Datasets/Original'
filesFound = glob.glob(in_dir + '/*.csv')
print('Extracting features from ' + str(len(filesFound)) + ' files...')
for currentFile in filesFound:
    print(currentFile)
    csv_df = pd.read_csv(currentFile)
    break
column_headers = csv_df.iloc[:,1:-2].columns.tolist()
csv_df

Extracting features from 98 files...
../Datasets/Original\11.4_Original_data.csv


,frame_number,"('nose', 'x')","('nose', 'y')","('nose', 'likelihood')","('H1R', 'x')","('H1R', 'y')","('H1R', 'likelihood')","('H2R', 'x')","('H2R', 'y')","('H2R', 'likelihood')",...,"('tail', 'y')","('tail', 'likelihood')","('S2', 'x')","('S2', 'y')","('S2', 'likelihood')","('S1', 'x')","('S1', 'y')","('S1', 'likelihood')",mouse_no,Freezing_label
0,0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,...,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,11.4,0
1,1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,...,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,11.4,0
2,2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,...,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,11.4,0
3,3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,...,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,11.4,0
4,4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,...,227.533890,0.999156,253.343109,281.232605,0.983985,304.960480,326.125854,0.995609,11.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,...,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,11.4,0
11246,11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,...,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,11.4,0
11247,11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,...,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,11.4,0
11248,11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,...,110.734329,0.999567,187.286819,111.964462,0.974826,226.513855,110.523743,0.992293,11.4,0


In [3]:
csv_df = csv_df.iloc[:,1:-2].copy()

In [4]:
def angle3pt(ax, ay, bx, by, cx, cy):
    ang = math.degrees(
        math.atan2(cy - by, cx - bx) - math.atan2(ay - by, ax - bx))
    return ang + 360 if ang < 0 else ang

In [5]:
columnHeaders = ["Mouse_nose_x", "Mouse_nose_y", "Mouse_nose_p", \
                         "Mouse_H1R_x", "Mouse_H1R_y", "Mouse_H1R_p", \
                         "Mouse_H2R_x", "Mouse_H2R_y", "Mouse_H2R_p", \
                         "Mouse_H1L_x", "Mouse_H1L_y", "Mouse_H1L_p", \
                         "Mouse_H2L_x", "Mouse_H2L_y", "Mouse_H2L_p", \
                         "Mouse_B1R_x", "Mouse_B1R_y", "Mouse_B1R_p", \
                         "Mouse_B2R_x", "Mouse_B2R_y", "Mouse_B2R_p", \
                         "Mouse_B3R_x", "Mouse_B3R_y", "Mouse_B3R_p", \
                         "Mouse_B1L_x", "Mouse_B1L_y", "Mouse_B1L_p", \
                         "Mouse_B2L_x", "Mouse_B2L_y", "Mouse_B2L_p", \
                         "Mouse_B3L_x", "Mouse_B3L_y", "Mouse_B3L_p", \
                         "Mouse_tail_x", "Mouse_tail_y", "Mouse_tail_p", \
                         "Mouse_S2_x", "Mouse_S2_y", "Mouse_S2_p", \
                         "Mouse_S1_x", "Mouse_S1_y", "Mouse_S1_p"]

csv_df.columns = columnHeaders

In [6]:
print('Evaluating convex hulls...')
########### MOUSE AREAS ###########################################
csv_df['Mouse_poly_area'] = csv_df.apply(lambda x: ConvexHull(np.array(
    [[x['Mouse_nose_x'], x["Mouse_nose_y"]],
     [x['Mouse_H1R_x'], x["Mouse_H1R_y"]],
     [x['Mouse_H2R_x'], x["Mouse_H2R_y"]],
     [x['Mouse_H1L_x'], x["Mouse_H1L_y"]], 
     [x['Mouse_H2L_x'], x["Mouse_H2L_y"]],
     [x['Mouse_B1R_x'], x["Mouse_B1R_y"]],
     [x['Mouse_B2R_x'], x["Mouse_B2R_y"]],
     [x['Mouse_B3R_x'], x["Mouse_B3R_y"]],
     [x['Mouse_B1L_x'], x["Mouse_B1L_y"]],
     [x['Mouse_B2L_x'], x["Mouse_B2L_y"]],
     [x['Mouse_B3L_x'], x["Mouse_B3L_y"]],
     [x['Mouse_tail_x'], x["Mouse_tail_y"]],
     [x['Mouse_S2_x'], x["Mouse_S2_y"]],
     [x['Mouse_S1_x'], x["Mouse_S1_y"]]])).area, axis=1)
csv_df

Evaluating convex hulls...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Mouse_tail_x,Mouse_tail_y,Mouse_tail_p,Mouse_S2_x,Mouse_S2_y,Mouse_S2_p,Mouse_S1_x,Mouse_S1_y,Mouse_S1_p,Mouse_poly_area
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,197.399582,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,599.294363
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,199.069702,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,602.073173
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,199.867844,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,603.542359
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,201.723312,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,607.245061
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,201.723312,227.533890,0.999156,253.343109,281.232605,0.983985,304.960480,326.125854,0.995609,623.814152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,115.261879,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,573.924363
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,115.261879,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,586.139715
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,115.261879,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,588.099724
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,115.261879,110.734329,0.999567,187.286819,111.964462,0.974826,226.513855,110.523743,0.992293,587.833687


In [7]:
########### CREATE SHIFTED DATAFRAME FOR DISTANCE CALCULATIONS ###########################################
csv_df_shifted = csv_df.shift(periods=1)
csv_df_shifted = csv_df_shifted.rename(
    columns = {
    "Mouse_nose_x": "Mouse_nose_x_shifted", "Mouse_nose_y": "Mouse_nose_y_shifted", "Mouse_nose_p": "Mouse_nose_p_shifted",
    "Mouse_H1R_x": "Mouse_H1R_x_shifted", "Mouse_H1R_y": "Mouse_H1R_y_shifted", "Mouse_H1R_p": "Mouse_H1R_p_shifted",
    "Mouse_H2R_x": "Mouse_H2R_x_shifted", "Mouse_H2R_y": "Mouse_H2R_y_shifted", "Mouse_H2R_p": "Mouse_H2R_p_shifted",
    "Mouse_H1L_x": "Mouse_H1L_x_shifted", "Mouse_H1L_y": "Mouse_H1L_y_shifted", "Mouse_H1L_p": "Mouse_H1L_p_shifted",
    "Mouse_H2L_x": "Mouse_H2L_x_shifted", "Mouse_H2L_y": "Mouse_H2L_y_shifted", "Mouse_H2L_p": "Mouse_H2L_p_shifted",
    "Mouse_B1R_x": "Mouse_B1R_x_shifted", "Mouse_B1R_y": "Mouse_B1R_y_shifted", "Mouse_B1R_p": "Mouse_B1R_p_shifted",
    "Mouse_B2R_x": "Mouse_B2R_x_shifted", "Mouse_B2R_y": "Mouse_B2R_y_shifted", "Mouse_B2R_p": "Mouse_B2R_p_shifted",
    "Mouse_B3R_x": "Mouse_B3R_x_shifted", "Mouse_B3R_y": "Mouse_B3R_y_shifted", "Mouse_B3R_p": "Mouse_B3R_p_shifted",
    "Mouse_B1L_x": "Mouse_B1L_x_shifted", "Mouse_B1L_y": "Mouse_B1L_y_shifted", "Mouse_B1L_p": "Mouse_B1L_p_shifted",
    "Mouse_B2L_x": "Mouse_B2L_x_shifted", "Mouse_B2L_y": "Mouse_B2L_y_shifted", "Mouse_B2L_p": "Mouse_B2L_p_shifted",
    "Mouse_B3L_x": "Mouse_B3L_x_shifted", "Mouse_B3L_y": "Mouse_B3L_y_shifted", "Mouse_B3L_p": "Mouse_B3L_p_shifted",
    "Mouse_tail_x": "Mouse_tail_x_shifted", "Mouse_tail_y": "Mouse_tail_y_shifted", "Mouse_tail_p": "Mouse_tail_p_shifted",
    "Mouse_S2_x": "Mouse_S2_x_shifted", "Mouse_S2_y": "Mouse_S2_y_shifted", "Mouse_S2_p": "Mouse_S2_p_shifted",
    "Mouse_S1_x": "Mouse_S1_x_shifted", "Mouse_S1_y": "Mouse_S1_y_shifted", "Mouse_S1_p": "Mouse_S1_p_shifted",
    "Mouse_poly_area":"Mouse_poly_area_shifted"
})

csv_df_combined = pd.concat([csv_df, csv_df_shifted], axis=1, join='inner')
csv_df_combined = csv_df_combined.fillna(0)
csv_df_combined = csv_df_combined.reset_index(drop=True)
csv_df_combined

,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Mouse_tail_x_shifted,Mouse_tail_y_shifted,Mouse_tail_p_shifted,Mouse_S2_x_shifted,Mouse_S2_y_shifted,Mouse_S2_p_shifted,Mouse_S1_x_shifted,Mouse_S1_y_shifted,Mouse_S1_p_shifted,Mouse_poly_area_shifted
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,197.399582,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,599.294363
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,199.069702,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,602.073173
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,199.867844,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,603.542359
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,201.723312,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,607.245061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,115.967751,108.480583,0.996206,191.435760,106.086227,0.988119,236.414139,105.547989,0.990182,569.459741
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,115.261879,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,573.924363
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,115.261879,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,586.139715
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,115.261879,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,588.099724


In [8]:
print('Calculating euclidean distances...')
########### EUCLIDEAN DISTANCES ###########################################
# Head to Tail
csv_df['Nose_to_tail'] = np.sqrt((csv_df.Mouse_nose_x - csv_df.Mouse_tail_x) ** 2 + (csv_df.Mouse_nose_y - csv_df.Mouse_tail_y) ** 2)

# Body distances
csv_df['Distance_body_S1_S2'] = np.sqrt((csv_df.Mouse_S1_x - csv_df.Mouse_S2_x) ** 2 + (csv_df.Mouse_S1_y - csv_df.Mouse_S2_y) ** 2)

csv_df['Distance_body_H1L_B1L'] = np.sqrt((csv_df.Mouse_H1L_x - csv_df.Mouse_B1L_x) ** 2 + (csv_df.Mouse_H1L_y - csv_df.Mouse_B1L_y) ** 2)
csv_df['Distance_body_H1R_B1R'] = np.sqrt((csv_df.Mouse_H1R_x - csv_df.Mouse_B1R_x) ** 2 + (csv_df.Mouse_H1R_y - csv_df.Mouse_B1R_y) ** 2)
csv_df['Distance_body_H2L_B2L'] = np.sqrt((csv_df.Mouse_H2L_x - csv_df.Mouse_B2L_x) ** 2 + (csv_df.Mouse_H2L_y - csv_df.Mouse_B2L_y) ** 2)
csv_df['Distance_body_H2R_B2R'] = np.sqrt((csv_df.Mouse_H2R_x - csv_df.Mouse_B2R_x) ** 2 + (csv_df.Mouse_H2R_y - csv_df.Mouse_B2R_y) ** 2)
csv_df['Distance_body_B1L_B2L'] = np.sqrt((csv_df.Mouse_B1L_x - csv_df.Mouse_B2L_x) ** 2 + (csv_df.Mouse_B1L_y - csv_df.Mouse_B2L_y) ** 2)
csv_df['Distance_body_B1R_B2R'] = np.sqrt((csv_df.Mouse_B1R_x - csv_df.Mouse_B2R_x) ** 2 + (csv_df.Mouse_B1R_y - csv_df.Mouse_B2R_y) ** 2)
csv_df['Distance_body_H1L_H2L'] = np.sqrt((csv_df.Mouse_H1L_x - csv_df.Mouse_H2L_x) ** 2 + (csv_df.Mouse_H1L_y - csv_df.Mouse_H2L_y) ** 2)
csv_df['Distance_body_H1R_H2R'] = np.sqrt((csv_df.Mouse_H1R_x - csv_df.Mouse_H2R_x) ** 2 + (csv_df.Mouse_H1R_y - csv_df.Mouse_H2R_y) ** 2)
csv_df['Distance_body_B2L_B3L'] = np.sqrt((csv_df.Mouse_B2L_x - csv_df.Mouse_B3L_x) ** 2 + (csv_df.Mouse_B2L_y - csv_df.Mouse_B3L_y) ** 2)
csv_df['Distance_body_B2R_B3R'] = np.sqrt((csv_df.Mouse_B2R_x - csv_df.Mouse_B3R_x) ** 2 + (csv_df.Mouse_B2R_y - csv_df.Mouse_B3R_y) ** 2)
csv_df['Distance_body_B1L_B3L'] = np.sqrt((csv_df.Mouse_B1L_x - csv_df.Mouse_B3L_x) ** 2 + (csv_df.Mouse_B1L_y - csv_df.Mouse_B3L_y) ** 2)
csv_df['Distance_body_B1R_B3R'] = np.sqrt((csv_df.Mouse_B1R_x - csv_df.Mouse_B3R_x) ** 2 + (csv_df.Mouse_B1R_y - csv_df.Mouse_B3R_y) ** 2)

csv_df['Distance_bilateral_B1L_B3R'] = np.sqrt((csv_df.Mouse_B1L_x - csv_df.Mouse_B3R_x) ** 2 + (csv_df.Mouse_B1L_y - csv_df.Mouse_B3R_y) ** 2)
csv_df['Distance_bilateral_B1R_B3L'] = np.sqrt((csv_df.Mouse_B1R_x - csv_df.Mouse_B3L_x) ** 2 + (csv_df.Mouse_B1R_y - csv_df.Mouse_B3L_y) ** 2)

# Back distances
csv_df['Distance_back_tail'] = np.sqrt((csv_df.Mouse_S2_x - csv_df.Mouse_tail_x) ** 2 + (csv_df.Mouse_S2_y - csv_df.Mouse_tail_y) ** 2)
csv_df['Distance_back_nose'] = np.sqrt((csv_df.Mouse_S2_x - csv_df.Mouse_nose_x) ** 2 + (csv_df.Mouse_S2_y - csv_df.Mouse_nose_y) ** 2)

# Movements
csv_df['Movement_nose'] = np.sqrt((csv_df_combined.Mouse_tail_x_shifted - csv_df_combined.Mouse_tail_x) ** 2 + (csv_df_combined.Mouse_tail_y_shifted - csv_df_combined.Mouse_tail_y) ** 2)
csv_df['Movement_tail'] = np.sqrt((csv_df_combined.Mouse_S2_x_shifted - csv_df_combined.Mouse_S2_x) ** 2 + (csv_df_combined.Mouse_S2_y_shifted - csv_df_combined.Mouse_S2_y) ** 2)

csv_df['Movement_H1L'] = np.sqrt((csv_df_combined.Mouse_H1L_x_shifted - csv_df_combined.Mouse_H1L_x) ** 2 + (csv_df_combined.Mouse_H1L_y_shifted - csv_df_combined.Mouse_H1L_y) ** 2)
csv_df['Movement_H1R'] = np.sqrt((csv_df_combined.Mouse_H1R_x_shifted - csv_df_combined.Mouse_H1R_x) ** 2 + (csv_df_combined.Mouse_H1R_y_shifted - csv_df_combined.Mouse_H1R_y) ** 2)
csv_df['Movement_H2L'] = np.sqrt((csv_df_combined.Mouse_H2L_x_shifted - csv_df_combined.Mouse_H2L_x) ** 2 + (csv_df_combined.Mouse_H2L_y_shifted - csv_df_combined.Mouse_H2L_y) ** 2)
csv_df['Movement_H2R'] = np.sqrt((csv_df_combined.Mouse_H2R_x_shifted - csv_df_combined.Mouse_H2R_x) ** 2 + (csv_df_combined.Mouse_H2R_y_shifted - csv_df_combined.Mouse_H2R_y) ** 2)
csv_df['Movement_B1L'] = np.sqrt((csv_df_combined.Mouse_B1L_x_shifted - csv_df_combined.Mouse_B1L_x) ** 2 + (csv_df_combined.Mouse_B1L_y_shifted - csv_df_combined.Mouse_B1L_y) ** 2)
csv_df['Movement_B1R'] = np.sqrt((csv_df_combined.Mouse_B1R_x_shifted - csv_df_combined.Mouse_B1R_x) ** 2 + (csv_df_combined.Mouse_B1R_y_shifted - csv_df_combined.Mouse_B1R_y) ** 2)
csv_df['Movement_B2L'] = np.sqrt((csv_df_combined.Mouse_B2L_x_shifted - csv_df_combined.Mouse_B2L_x) ** 2 + (csv_df_combined.Mouse_B2L_y_shifted - csv_df_combined.Mouse_B2L_y) ** 2)
csv_df['Movement_B2R'] = np.sqrt((csv_df_combined.Mouse_B2R_x_shifted - csv_df_combined.Mouse_B2R_x) ** 2 + (csv_df_combined.Mouse_B2R_y_shifted - csv_df_combined.Mouse_B2R_y) ** 2)
csv_df['Movement_B3L'] = np.sqrt((csv_df_combined.Mouse_B3L_x_shifted - csv_df_combined.Mouse_B3L_x) ** 2 + (csv_df_combined.Mouse_B3L_y_shifted - csv_df_combined.Mouse_B3L_y) ** 2)
csv_df['Movement_B3R'] = np.sqrt((csv_df_combined.Mouse_B3R_x_shifted - csv_df_combined.Mouse_B3R_x) ** 2 + (csv_df_combined.Mouse_B3R_y_shifted - csv_df_combined.Mouse_B3R_y) ** 2)

# Movements for center points
csv_df['Movement_S1'] = np.sqrt((csv_df_combined.Mouse_S1_x_shifted - csv_df_combined.Mouse_S1_x) ** 2 + (csv_df_combined.Mouse_S1_y_shifted - csv_df_combined.Mouse_S1_y) ** 2)
csv_df['Movement_S2'] = np.sqrt((csv_df_combined.Mouse_S2_x_shifted - csv_df_combined.Mouse_S2_x) ** 2 + (csv_df_combined.Mouse_S2_y_shifted - csv_df_combined.Mouse_S2_y) ** 2)

csv_df['Mouse_polygon_size_change'] = csv_df_combined.Mouse_poly_area_shifted - csv_df_combined.Mouse_poly_area
csv_df

Calculating euclidean distances...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Movement_H2R,Movement_B1L,Movement_B1R,Movement_B2L,Movement_B2R,Movement_B3L,Movement_B3R,Movement_S1,Movement_S2,Mouse_polygon_size_change
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,525.934460,478.832141,462.707666,360.146257,354.488170,317.669783,309.534130,432.325910,374.270084,-599.294363
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,6.225722,3.576042,2.020613,0.993107,0.532200,0.867740,0.778271,4.463065,0.599896,-2.778810
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,2.060988,3.092987,3.425495,1.128929,0.796892,0.525973,2.658180,4.184567,2.681183,-1.469186
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,4.751312,3.057615,9.886567,2.634120,0.446943,6.314864,0.793884,4.597674,2.978293,-3.702703
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,17.283508,1.897766,3.866761,1.244858,7.973737,4.321640,5.926804,1.824449,0.000000,-16.569091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,11.796469,1.923994,4.199890,2.358407,4.116257,2.133322,0.346242,7.085832,0.812447,-4.464622
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,11.517721,15.925198,6.497114,10.424437,3.501787,5.191569,7.936983,0.524925,3.018843,-12.215352
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,2.092857,3.221604,3.592963,0.413666,0.816766,1.721782,1.472280,1.038376,4.143212,-1.960009
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,5.037170,4.102816,0.623383,0.284500,0.385483,0.696915,0.335419,3.690277,0.953018,0.266037


In [9]:
# Create lists to store calculated statistics
large_euclidean_list = []
small_euclidean_list = []
mean_euclidean_list = []
sum_euclidean_list = []

# Loop over DataFrame rows
for index, row in csv_df.iterrows():
    # Create NumPy array for points
    body_parts_np_array = np.array([
        [row['Mouse_nose_x'], row['Mouse_nose_y']],
        [row['Mouse_H1R_x'], row['Mouse_H1R_y']],
        [row['Mouse_H2R_x'], row['Mouse_H2R_y']],
        [row['Mouse_H1L_x'], row['Mouse_H1L_y']],
        [row['Mouse_H2L_x'], row['Mouse_H2L_y']],
        [row['Mouse_B1R_x'], row['Mouse_B1R_y']],
        [row['Mouse_B2R_x'], row['Mouse_B2R_y']],
        [row['Mouse_B3R_x'], row['Mouse_B3R_y']],
        [row['Mouse_B1L_x'], row['Mouse_B1L_y']],
        [row['Mouse_B2L_x'], row['Mouse_B2L_y']],
        [row['Mouse_B3L_x'], row['Mouse_B3L_y']],
        [row['Mouse_tail_x'], row['Mouse_tail_y']],
        [row['Mouse_S2_x'], row['Mouse_S2_y']],
        [row['Mouse_S1_x'], row['Mouse_S1_y']]
    ]).astype(int)

    # Calculate Euclidean distances
    dist_euclidean = scipy.spatial.distance.cdist(body_parts_np_array, body_parts_np_array, metric='euclidean')

    # Extract statistics from distances
    large_euclidean = np.amax(dist_euclidean)
    small_euclidean = np.min(dist_euclidean)
    mean_euclidean = np.mean(dist_euclidean)
    sum_euclidean = np.sum(dist_euclidean)

    # Append statistics to lists
    large_euclidean_list.append(large_euclidean)
    small_euclidean_list.append(small_euclidean)
    mean_euclidean_list.append(mean_euclidean)
    sum_euclidean_list.append(sum_euclidean)

# Create new columns in DataFrame
csv_df['Largest_euclidean_distance_hull'] = large_euclidean_list
csv_df['Smallest_euclidean_distance_hull'] = small_euclidean_list
csv_df['Mean_euclidean_distance_hull'] = mean_euclidean_list
csv_df['Sum_euclidean_distance_hull'] = sum_euclidean_list
csv_df

,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Movement_B2R,Movement_B3L,Movement_B3R,Movement_S1,Movement_S2,Mouse_polygon_size_change,Largest_euclidean_distance_hull,Smallest_euclidean_distance_hull,Mean_euclidean_distance_hull,Sum_euclidean_distance_hull
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,354.488170,317.669783,309.534130,432.325910,374.270084,-599.294363,257.775096,0.0,119.348645,23392.334416
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,0.532200,0.867740,0.778271,4.463065,0.599896,-2.778810,258.373760,0.0,120.031794,23526.231719
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,0.796892,0.525973,2.658180,4.184567,2.681183,-1.469186,260.403533,0.0,120.636674,23644.788150
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,0.446943,6.314864,0.793884,4.597674,2.978293,-3.702703,263.503321,0.0,121.280051,23770.889930
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,7.973737,4.321640,5.926804,1.824449,0.000000,-16.569091,273.305690,0.0,124.949258,24490.054575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,4.116257,2.133322,0.346242,7.085832,0.812447,-4.464622,252.192387,0.0,110.735352,21704.128984
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,3.501787,5.191569,7.936983,0.524925,3.018843,-12.215352,260.693306,0.0,113.575280,22260.754782
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,0.816766,1.721782,1.472280,1.038376,4.143212,-1.960009,262.093495,0.0,114.464716,22435.084242
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,0.385483,0.696915,0.335419,3.690277,0.953018,0.266037,262.017175,0.0,114.610056,22463.570984


In [10]:
# Create a new column for total movement
movement_columns = [
    'Movement_nose',
    'Movement_tail',
    'Movement_H1L',
    'Movement_H1R',
    'Movement_H2L',
    'Movement_H2R',
    'Movement_B1L',
    'Movement_B1R',
    'Movement_B2L',
    'Movement_B2R',
    'Movement_B3L',
    'Movement_B3R',
    'Movement_S1',
    'Movement_S2'
]

csv_df['Total_movement_all_bodyparts'] = csv_df[movement_columns].sum(axis=1)
csv_df

,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Movement_B3L,Movement_B3R,Movement_S1,Movement_S2,Mouse_polygon_size_change,Largest_euclidean_distance_hull,Smallest_euclidean_distance_hull,Mean_euclidean_distance_hull,Sum_euclidean_distance_hull,Total_movement_all_bodyparts
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,317.669783,309.534130,432.325910,374.270084,-599.294363,257.775096,0.0,119.348645,23392.334416,5894.581468
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,0.867740,0.778271,4.463065,0.599896,-2.778810,258.373760,0.0,120.031794,23526.231719,30.899253
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,0.525973,2.658180,4.184567,2.681183,-1.469186,260.403533,0.0,120.636674,23644.788150,33.802594
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,6.314864,0.793884,4.597674,2.978293,-3.702703,263.503321,0.0,121.280051,23770.889930,58.187067
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,4.321640,5.926804,1.824449,0.000000,-16.569091,273.305690,0.0,124.949258,24490.054575,95.023895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,2.133322,0.346242,7.085832,0.812447,-4.464622,252.192387,0.0,110.735352,21704.128984,80.619699
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,5.191569,7.936983,0.524925,3.018843,-12.215352,260.693306,0.0,113.575280,22260.754782,113.803291
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,1.721782,1.472280,1.038376,4.143212,-1.960009,262.093495,0.0,114.464716,22435.084242,40.843219
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,0.696915,0.335419,3.690277,0.953018,0.266037,262.017175,0.0,114.610056,22463.570984,36.407896


In [11]:
print('Calculating angles...')
csv_df['Mouse_angle'] = csv_df.apply(lambda x: angle3pt(x['Mouse_nose_x'], x['Mouse_nose_y'], x['Mouse_S2_x'], x['Mouse_S2_y'], x['Mouse_tail_x'], x['Mouse_tail_y']), axis=1)
csv_df

Calculating angles...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Movement_B3R,Movement_S1,Movement_S2,Mouse_polygon_size_change,Largest_euclidean_distance_hull,Smallest_euclidean_distance_hull,Mean_euclidean_distance_hull,Sum_euclidean_distance_hull,Total_movement_all_bodyparts,Mouse_angle
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,309.534130,432.325910,374.270084,-599.294363,257.775096,0.0,119.348645,23392.334416,5894.581468,193.650302
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,0.778271,4.463065,0.599896,-2.778810,258.373760,0.0,120.031794,23526.231719,30.899253,193.682985
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,2.658180,4.184567,2.681183,-1.469186,260.403533,0.0,120.636674,23644.788150,33.802594,190.580005
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,0.793884,4.597674,2.978293,-3.702703,263.503321,0.0,121.280051,23770.889930,58.187067,190.004062
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,5.926804,1.824449,0.000000,-16.569091,273.305690,0.0,124.949258,24490.054575,95.023895,190.583435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,0.346242,7.085832,0.812447,-4.464622,252.192387,0.0,110.735352,21704.128984,80.619699,164.438320
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,7.936983,0.524925,3.018843,-12.215352,260.693306,0.0,113.575280,22260.754782,113.803291,172.188302
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,1.472280,1.038376,4.143212,-1.960009,262.093495,0.0,114.464716,22435.084242,40.843219,179.352798
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,0.335419,3.690277,0.953018,0.266037,262.017175,0.0,114.610056,22463.570984,36.407896,180.407088


In [12]:
print('Calculating rolling windows descriptives: medians, medians, and sums...')
    
for column in [
    'Nose_to_tail',
    'Distance_body_S1_S2',
    'Distance_body_H1L_B1L',
    'Distance_body_H1R_B1R',
    'Distance_body_H2L_B2L',
    'Distance_body_H2R_B2R',
    'Distance_body_B1L_B2L',
    'Distance_body_B1R_B2R',
    'Distance_body_H1L_H2L',
    'Distance_body_H1R_H2R',
    'Distance_body_B2L_B3L',
    'Distance_body_B2R_B3R',
    'Distance_body_B1L_B3L',
    'Distance_body_B1R_B3R',
    'Distance_bilateral_B1L_B3R',
    'Distance_bilateral_B1R_B3L',
    'Distance_back_tail',
    'Distance_back_nose',
    'Movement_nose',
    'Movement_tail',
    'Movement_H1L',
    'Movement_H1R',
    'Movement_H2L',
    'Movement_H2R',
    'Movement_B1L',
    'Movement_B1R',
    'Movement_B2L',
    'Movement_B2R',
    'Movement_B3L',
    'Movement_B3R',
    'Movement_S1',
    'Movement_S2',
    'Total_movement_all_bodyparts',
    'Largest_euclidean_distance_hull',
    'Smallest_euclidean_distance_hull',
    'Mean_euclidean_distance_hull',
]:
    temp_df = pd.DataFrame()
    current_col_name_median = f'{column}_median_25'
    current_col_name_mean = f'{column}_mean_25'
    current_col_name_sum = f'{column}_sum_25'

    temp_df[current_col_name_median] = csv_df[column].rolling(25, min_periods=1).median()
    temp_df[current_col_name_mean] = csv_df[column].rolling(25, min_periods=1).mean()
    temp_df[current_col_name_sum] = csv_df[column].rolling(25, min_periods=1).sum()

    csv_df = pd.concat([csv_df, temp_df], axis=1)

csv_df

Calculating rolling windows descriptives: medians, medians, and sums...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Total_movement_all_bodyparts_sum_25,Largest_euclidean_distance_hull_median_25,Largest_euclidean_distance_hull_mean_25,Largest_euclidean_distance_hull_sum_25,Smallest_euclidean_distance_hull_median_25,Smallest_euclidean_distance_hull_mean_25,Smallest_euclidean_distance_hull_sum_25,Mean_euclidean_distance_hull_median_25,Mean_euclidean_distance_hull_mean_25,Mean_euclidean_distance_hull_sum_25
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,5894.581468,257.775096,257.775096,257.775096,0.0,0.0,0.0,119.348645,119.348645,119.348645
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,5925.480721,258.074428,258.074428,516.148856,0.0,0.0,0.0,119.690220,119.690220,239.380439
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,5959.283315,258.373760,258.850796,776.552389,0.0,0.0,0.0,120.031794,120.005705,360.017114
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,6017.470382,259.388647,260.013927,1040.055710,0.0,0.0,0.0,120.334234,120.324291,481.297164
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,6112.494277,260.403533,262.672280,1313.361399,0.0,0.0,0.0,120.636674,121.249284,606.246422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,371.304581,223.271136,225.911912,5647.797792,0.0,0.0,0.0,103.598845,104.234776,2605.869399
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,481.290128,223.271136,227.370987,5684.274685,0.0,0.0,0.0,103.598845,104.631023,2615.775580
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,519.722372,223.271136,228.936880,5723.421998,0.0,0.0,0.0,103.600215,105.069377,2626.734415
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,555.036230,224.216413,230.499720,5762.492991,0.0,0.0,0.0,103.775395,105.515255,2637.881376


In [13]:
print('Calculating deviations...')
csv_df['Total_movement_all_bodyparts_both_deviation'] = (csv_df['Total_movement_all_bodyparts'].mean() - csv_df['Total_movement_all_bodyparts'])
csv_df['Smallest_euclid_distances_hull_deviation'] = (csv_df['Smallest_euclidean_distance_hull'].mean() - csv_df['Smallest_euclidean_distance_hull'])
csv_df['Largest_euclid_distances_hull_deviation'] = (csv_df['Largest_euclidean_distance_hull'].mean() - csv_df['Largest_euclidean_distance_hull'])
csv_df['Mean_euclid_distances_hull_deviation'] = (csv_df['Mean_euclidean_distance_hull'].mean() - csv_df['Mean_euclidean_distance_hull'])
csv_df['Movement_deviation_S2'] = (csv_df['Movement_S2'].mean() - csv_df['Movement_S2'])
csv_df['Polygon_deviation'] = (csv_df['Mouse_poly_area'].mean() - csv_df['Mouse_poly_area'])

csv_df['Smallest_euclidean_distances_hull_mean_25_deviation'] = (csv_df['Smallest_euclidean_distance_hull_mean_25'].mean() - csv_df['Smallest_euclidean_distance_hull_mean_25'])
csv_df['Largest_euclidean_distance_hull_mean_25_deviation'] = (csv_df['Largest_euclidean_distance_hull_mean_25'].mean() - csv_df['Largest_euclidean_distance_hull_mean_25'])
csv_df['Mean_euclidean_distance_hull_mean_25_deviation'] = (csv_df['Mean_euclidean_distance_hull_mean_25'].mean() - csv_df['Mean_euclidean_distance_hull_mean_25'])
csv_df['Total_movement_all_bodyparts_mean_25_deviation'] = (csv_df['Total_movement_all_bodyparts_mean_25'].mean() - csv_df['Total_movement_all_bodyparts_mean_25'])
csv_df

Calculating deviations...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Total_movement_all_bodyparts_both_deviation,Smallest_euclid_distances_hull_deviation,Largest_euclid_distances_hull_deviation,Mean_euclid_distances_hull_deviation,Movement_deviation_S2,Polygon_deviation,Smallest_euclidean_distances_hull_mean_25_deviation,Largest_euclidean_distance_hull_mean_25_deviation,Mean_euclidean_distance_hull_mean_25_deviation,Total_movement_all_bodyparts_mean_25_deviation
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,-5814.056519,0.0,12.449082,-2.110039,-369.992090,40.418581,0.0,12.491039,-2.090336,-5812.543194
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,49.625696,0.0,11.850418,-2.793188,3.678098,37.639771,0.0,12.191707,-2.431911,-2880.702087
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,46.722355,0.0,9.820645,-3.398068,1.596811,36.170585,0.0,11.415339,-2.747396,-1904.389498
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,22.337882,0.0,6.720858,-4.041444,1.299701,32.467882,0.0,10.252208,-3.065982,-1422.329322
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,-14.498946,0.0,-3.081511,-7.710652,4.277994,15.898792,0.0,7.593855,-3.990976,-1140.460582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,-0.094750,0.0,18.031791,6.503254,3.465547,65.788581,0.0,44.354223,13.023533,67.186090
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,-33.278342,0.0,9.530872,3.663327,1.259151,53.573229,0.0,42.895148,12.627286,62.786668
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,39.681730,0.0,8.130683,2.773891,0.134782,51.613219,0.0,41.329255,12.188932,61.249378
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,44.117053,0.0,8.207003,2.628550,3.324976,51.879256,0.0,39.766415,11.743054,59.836824


In [20]:
print('Calculating percentile ranks...')
csv_df['Movement_percentile_rank'] = csv_df['Movement_S2'].rank(pct=True)

csv_df['Mean_euclidean_distance_hull_mean_25_percentile_rank'] = csv_df['Mean_euclidean_distance_hull_mean_25'].rank(pct=True)
csv_df['Smallest_euclidean_distance_hull_mean_25_percentile_rank'] = csv_df['Smallest_euclidean_distance_hull_mean_25'].rank(pct=True)
csv_df['Largest_euclidean_distance_hull_mean_25_percentile_rank'] = csv_df['Largest_euclidean_distance_hull_mean_25'].rank(pct=True)
csv_df['Total_movement_all_bodyparts_mean_25_percentile_rank'] = csv_df['Total_movement_all_bodyparts_mean_25'].rank(pct=True)
csv_df

Calculating percentile ranks...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Polygon_deviation,Smallest_euclidean_distances_hull_mean_25_deviation,Largest_euclidean_distance_hull_mean_25_deviation,Mean_euclidean_distance_hull_mean_25_deviation,Total_movement_all_bodyparts_mean_25_deviation,Movement_percentile_rank,Mean_euclidean_distance_hull_mean_25_percentile_rank,Smallest_euclidean_distance_hull_mean_25_percentile_rank,Largest_euclidean_distance_hull_mean_25_percentile_rank,Total_movement_all_bodyparts_mean_25_percentile_rank
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,40.418581,0.0,12.491039,-2.090336,-5812.543194,0.999733,0.627644,0.500044,0.512000,1.000000
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,37.639771,0.0,12.191707,-2.431911,-2880.702087,0.417422,0.636000,0.500044,0.515022,0.999911
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,36.170585,0.0,11.415339,-2.747396,-1904.389498,0.690311,0.642133,0.500044,0.521867,0.999822
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,32.467882,0.0,10.252208,-3.065982,-1422.329322,0.710133,0.649778,0.500044,0.531911,0.999733
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,15.898792,0.0,7.593855,-3.990976,-1140.460582,0.044044,0.673778,0.500044,0.557600,0.999644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,65.788581,0.0,44.354223,13.023533,67.186090,0.470756,0.202667,0.500044,0.212178,0.111644
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,53.573229,0.0,42.895148,12.627286,62.786668,0.713689,0.211022,0.500044,0.226133,0.144711
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,51.613219,0.0,41.329255,12.188932,61.249378,0.771556,0.220889,0.500044,0.239378,0.152711
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,51.879256,0.0,39.766415,11.743054,59.836824,0.499378,0.231022,0.500044,0.253511,0.162133


In [ ]:
# After engineering all features, drop the original coordinates (x, y, and p)
# We want to remove redundant features and leave around 20-30
# 

In [22]:
csv_df.columns.tolist()

['Mouse_nose_x',
 'Mouse_nose_y',
 'Mouse_nose_p',
 'Mouse_H1R_x',
 'Mouse_H1R_y',
 'Mouse_H1R_p',
 'Mouse_H2R_x',
 'Mouse_H2R_y',
 'Mouse_H2R_p',
 'Mouse_H1L_x',
 'Mouse_H1L_y',
 'Mouse_H1L_p',
 'Mouse_H2L_x',
 'Mouse_H2L_y',
 'Mouse_H2L_p',
 'Mouse_B1R_x',
 'Mouse_B1R_y',
 'Mouse_B1R_p',
 'Mouse_B2R_x',
 'Mouse_B2R_y',
 'Mouse_B2R_p',
 'Mouse_B3R_x',
 'Mouse_B3R_y',
 'Mouse_B3R_p',
 'Mouse_B1L_x',
 'Mouse_B1L_y',
 'Mouse_B1L_p',
 'Mouse_B2L_x',
 'Mouse_B2L_y',
 'Mouse_B2L_p',
 'Mouse_B3L_x',
 'Mouse_B3L_y',
 'Mouse_B3L_p',
 'Mouse_tail_x',
 'Mouse_tail_y',
 'Mouse_tail_p',
 'Mouse_S2_x',
 'Mouse_S2_y',
 'Mouse_S2_p',
 'Mouse_S1_x',
 'Mouse_S1_y',
 'Mouse_S1_p',
 'Mouse_poly_area',
 'Nose_to_tail',
 'Distance_body_S1_S2',
 'Distance_body_H1L_B1L',
 'Distance_body_H1R_B1R',
 'Distance_body_H2L_B2L',
 'Distance_body_H2R_B2R',
 'Distance_body_B1L_B2L',
 'Distance_body_B1R_B2R',
 'Distance_body_H1L_H2L',
 'Distance_body_H1R_H2R',
 'Distance_body_B2L_B3L',
 'Distance_body_B2R_B3R',
 'Dista